## Introduction 

This lesson will cover: 
- What function calling is and when to use it 
- How to create a function call with Azure OpenAI 
- How to integrate a function call into an application 

## Learning Goals 

After completing this lesson, you will know how to and understand: 

- The purpose of using function calling 
- How to set up Function Call using the Azure OpenAI Service 
- How to design effective function calls for your application's use case


## Understanding Function Calls 

For this lesson, we want to build a feature for our education startup that allows users to use a chatbot to find technical courses. We will recommend courses that fit their skill level, current role and technology of interest. 

To complete this we will use a combination of: 
 - `Azure Open AI` to create a chat experience for the user
 - `Microsoft Learn Catalog API` to help users find courses based on the request of the user 
 - `Function Calling` to take the user's query and send it to a function to make the API request. 

To get started, let's look at why we would want to use function calling in the first place: 

print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)


### Why Function Calling

If you have completed any other lesson in this course, you probably understand the power of using Large Language Models (LLMs). Hopefully, you can also see some of their limitations.

Function Calling is a feature of the Azure Open AI Service designed to address the following limitations:
1) Consistent response format
2) The ability to use data from other sources within an application's chat context

Before function calling, responses from an LLM were unstructured and inconsistent. Developers had to write complex validation code to handle every possible variation in a response.

Users couldn't get answers to questions like "What is the current weather in Stockholm?" because models were limited to the data available at the time they were trained.

Let's look at the example below to illustrate this problem:

Suppose we want to create a database of student data so we can recommend the right course to them. Below, we have two descriptions of students that are very similar in the data they contain.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

We want to send this to an LLM to parse the data. This can later be used in our application to send this to an API or store it in a database.

Let's create two identical prompts to instruct the LLM on what information we are interested in:


We want to send this to an LLM to parse the parts that are important to our product. So we can create two identical prompts to instruct the LLM:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


After creating these two prompts, we will send them to the LLM by using `openai.ChatCompletion`.  We store the prompt in the `messages` variable and assign the role to `user`. This is to mimic a message from a user being written to a chatbot.


In [ ]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-07-01-preview"
  )

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']

: 

Now we can send both requests to the LLM and examine the response we receive.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Even though the prompts are the same and the descriptions are similar, we can get different formats for the `Grades` property.

If you run the above cell multiple times, the format might be `3.7` or `3.7 GPA`.

This happens because the LLM takes unstructured data from the written prompt and also returns unstructured data. We need a structured format so we know what to expect when storing or using this data.

By using function calling, we can ensure that we get structured data back. When using function calling, the LLM doesn't actually call or execute any functions. Instead, we define a structure for the LLM to follow in its responses. We then use those structured responses to determine which function to run in our applications.


![Function Calling Flow Diagram](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.en.png)


### Use Cases for using function calls 

**Calling External Tools** 
Chatbots are excellent at answering users' questions. By using function calling, chatbots can use user messages to perform specific tasks. For example, a student might ask the chatbot to "Send email to my instructor saying I need more assistance with this subject." This could trigger a function call to `send_email(to: string, body: string)`

**Create API or Database Queries**
Users can search for information using natural language, which is then turned into a formatted query or API request. For instance, a teacher might ask, "Who are the students that completed the last assignment?" which could call a function named `get_completed(student_name: string, assignment: int, current_status: string)`

**Creating Structured Data**
Users can take a block of text or a CSV file and use the LLM to extract key information from it. For example, a student could turn a Wikipedia article about peace agreements into AI flash cards. This could be done using a function called `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Creating Your First Function Call

The process of creating a function call involves three main steps:
1. Call the Chat Completions API with a list of your functions and a user message
2. Read the model's response to determine what action to take, such as executing a function or making an API call
3. Make another call to the Chat Completions API with the response from your function, so that information can be used to generate a reply to the user.


![Flow of a Function Call](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.en.png)


### Elements of a function call

#### Users Input

The first step is to create a user message. This can be set dynamically by taking the value from a text input, or you can assign a value here directly. If this is your first time working with the Chat Completions API, we need to define the `role` and the `content` of the message.

The `role` can be either `system` (for setting rules), `assistant` (the model), or `user` (the end-user). For function calling, we will set this as `user` and provide an example question.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Creating functions.

Next, we will define a function and its parameters. Here, we will use just one function called `search_courses`, but you can create multiple functions.

**Important**: Functions are included in the system message to the LLM and will count towards your available token limit.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definitions**

`name` - The name of the function that we want to be called.

`description` - This is a description of how the function works. Here, it's important to be specific and clear.

`parameters` - A list of values and the format that you want the model to produce in its response.

`type` - The data type in which the properties will be stored.

`properties` - A list of the specific values that the model will use for its response.

`name` - The name of the property that the model will use in its formatted response.

`type` - The data type of this property.

`description` - Description of the specific property.

**Optional**

`required` - Required property for the function call to be completed.


### Making the function call 
After defining a function, we now need to include it in the call to the Chat Completion API. We do this by adding `functions` to the request. In this case, `functions=functions`.

There is also an option to set `function_call` to `auto`. This means we will let the LLM decide which function should be called based on the user message rather than assigning it ourselves.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Now let's look at the response and see how it is formatted:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

You can see that the function name is called, and from the user's message, the LLM was able to extract the data to fill in the function's arguments.


## 3. Integrating Function Calls into an Application.

After we've tested the formatted response from the LLM, we can now integrate it into an application.

### Managing the flow

To integrate this into our application, let's follow these steps:

First, let's make a call to the Open AI services and store the message in a variable called `response_message`.


In [ ]:
response_message = response.choices[0].message

Now we will define the function that will call the Microsoft Learn API to get a list of courses:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



As a best practice, we will then check if the model wants to call a function. After that, we will create one of the available functions and match it to the function being called.
Next, we will take the arguments of the function and map them to the arguments from the LLM.

Finally, we will append the function call message and the values returned by the `search_courses` message. This gives the LLM all the information it needs to
respond to the user in natural language.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Code Challenge 

Great work! To continue learning about Azure Open AI Function Calling, you can build: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst 
 - More parameters for the function that might help learners find more courses. You can find the available API parameters here: 
 - Create another function call that takes more information from the learner, such as their native language 
 - Add error handling for when the function call and/or API call does not return any suitable courses



---

**Disclaimer**:
This document has been translated using the AI translation service [Co-op Translator](https://github.com/Azure/co-op-translator). While we strive for accuracy, please be aware that automated translations may contain errors or inaccuracies. The original document in its native language should be considered the authoritative source. For critical information, professional human translation is recommended. We are not liable for any misunderstandings or misinterpretations arising from the use of this translation.
